In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('..')
from pipeline import *
from models import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def transform_data(data_paths, num_blocks=8, *args, **kwargs):
    pipeline = AABB245_Pipeline(num_blocks=num_blocks)
    dataset = PolymerDataset(data_paths, pipeline)
    return dataset

def build_model(dataset, *args, **kwargs):
    return VanillaLSTM(input_dim=dataset.num_features, output_dim=dataset.num_classes)

data_paths = ['../data/AA66266AA.npy', '../data/AA66466AA.npy']
param_grid = {
    'num_blocks': [4],
    'lr_rate': [0.01],
    'num_epochs': 10,
    'verbose': False,
    'log': False
}
params, metrics = grid_search_cv(data_paths=data_paths, param_grid=param_grid,
                                 model_fn=build_model, train_fn=train, test_fn=test, transform_fn=transform_data, k_fold=2)

Processing ../data/AA66466AA.npy: 100%|██████████| 20752/20752 [00:11<00:00, 1742.63it/s]


AttributeError: 'functools.partial' object has no attribute 'info'

In [ ]:
pipeline = AABB245_Pipeline(num_blocks=32)
dataset = PolymerDataset(['../data/AA66266AA.npy', '../data/AA66566AA.npy'], pipeline)

In [ ]:
dataset.data.shape, dataset.labels.shape

In [ ]:
train_data, test_data = train_test_split(dataset)

In [ ]:
model = VanillaLSTM(input_dim=dataset.num_features, output_dim=dataset.num_classes)

In [ ]:
model, metrics = train(train_dataset=train_data, test_dataset=test_data, model=model, batch_size=512, num_epochs=100, lr_rate=0.01)

In [ ]:
param_grid = {
    'data_paths': [['../data/AA66266AA.npy', '../data/AA66466AA.npy'],
                   ['../data/AA66266AA.npy', '../data/AA66566AA.npy'],
                   ['../data/AA66466AA.npy', '../data/AA66566AA.npy'],
                   ['../data/AA66266AA.npy', '../data/AA66466AA.npy', '../data/AA66566AA.npy']],
    'num_blocks': [4, 8, 16, 32, 64, 128],
    'lr_rate': [0.001, 0.01],
    'num_epochs': [100, 200]
}

run_245(param_grid)



In [ ]:
param_grid = {
    'data_paths': [['../data/AA66266AA.npy', '../data/AA66566AA.npy']],
    'num_blocks': [16, 32, 64, 128],
    'lr_rate': [0.001, 0.01],
    'num_epochs': [100, 200]
}

run_245(param_grid)

In [ ]:
param_grid = {
    'data_paths': [['../data/AA66466AA.npy', '../data/AA66566AA.npy'],
                   ['../data/AA66266AA.npy', '../data/AA66466AA.npy', '../data/AA66566AA.npy']],
    'num_blocks': [4, 8, 16, 32, 64, 128],
    'lr_rate': [0.001, 0.01],
    'num_epochs': [100, 200]
}

run_245(param_grid)